In [1]:
import requests
import time
import csv
import pandas as pd
import os
from web3 import Web3
from dotenv import load_dotenv

## Polygon

In [41]:
load_dotenv()
api_key = os.getenv('INFURA_API_KEY_3')

# infura_mainnet_url = f'https://mainnet.infura.io/v3/{api_key}'
infura_polygon_url = f'https://polygon-mainnet.infura.io/v3/{api_key}'
# infura_arbitrum_url = f'https://arbitrum-mainnet.infura.io/v3/{api_key}'

web3 = Web3(Web3.HTTPProvider(infura_polygon_url))

def is_contract(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return len(code) > 2

In [ ]:
csv1 = pd.read_csv('../data/addresses/checked_addresses_1_year_1.csv')
csv2 = pd.read_csv('../data/addresses/checked_addresses_1_year.csv')
csv3 = pd.read_csv('../data/addresses/node_table.csv')

In [ ]:
# Combine all DataFrames into one with an additional column indicating the source DataFrame
combined_df = pd.concat([
    csv1.assign(source='csv1'),
    csv2.assign(source='csv2'),
    csv3.assign(source='csv3')
])

# Group by address and check if there are conflicting types
conflicting_addresses = combined_df.groupby('address').filter(
    lambda group: group['type'].nunique() > 1 and group['source'].nunique() >= 2
)

# Extract and print the unique conflicting addresses
conflicting_address_list = conflicting_addresses['address'].unique()
print(conflicting_address_list)

In [ ]:
transfers_df = pd.read_csv('../data/1_year/reduced_transfers_1_year.csv')

main_tokens = ['AAVE', 'aEthWETH', 'COMP', 'cWETHv3']
arb_tokens = ['ArbAAVE', 'aArbWETH', 'ArbCOMP', 'cArbWETH']
pol_tokens = ['PolAAVE', 'aPolWETH', 'PolCOMP']

main_df = transfers_df[
    transfers_df['token'].isin(main_tokens) &
    (transfers_df['from'].str.lower() != transfers_df['to'].str.lower()) &
    (transfers_df['value'] != 0)
]

arb_df = transfers_df[
    transfers_df['token'].isin(arb_tokens) &
    (transfers_df['from'].str.lower() != transfers_df['to'].str.lower()) &
    (transfers_df['value'] != 0)
]

pol_df = transfers_df[
    transfers_df['token'].isin(pol_tokens) &
    (transfers_df['from'].str.lower() != transfers_df['to'].str.lower()) &
    (transfers_df['value'] != 0)
]

In [12]:
main_addresses = set(main_df['from'].unique()).union(set(main_df['to'].unique()))
arb_addresses = set(arb_df['from'].unique()).union(set(arb_df['to'].unique()))
pol_addresses = set(pol_df['from'].unique()).union(set(pol_df['to'].unique()))

In [20]:
csv_arb = pd.read_csv('../data/addresses/checked_arb_addresses.csv')
csv_pol = pd.read_csv('../data/addresses/checked_pol_addresses.csv')
csv_main = pd.read_csv('../data/addresses/checked_main_addresses.csv')

In [ ]:
# Merge the dataframes on the 'address' column
merged_df = pd.merge(csv_arb, csv_pol, on='address', suffixes=('_arb', '_pol'), how='inner')
merged_df = pd.merge(merged_df, csv_main, on='address', suffixes=('', '_main'))

# Find rows where the 'type' field isn't the same across the dataframes
conflicting_types = merged_df[
    (merged_df['type_arb'] != merged_df['type_pol']) |
    (merged_df['type_arb'] != merged_df['type']) |
    (merged_df['type_pol'] != merged_df['type'])
]

# Print the conflicting elements
print(conflicting_types)

In [ ]:
combined_csv = pd.concat([csv_arb, csv_pol, csv_main])
combined_csv = combined_csv.groupby('address', as_index=False).agg({'type': 'any'})
combined_csv.to_csv('combined_addresses.csv', index=False)

print("Combined CSV saved as 'combined_addresses.csv'")

In [ ]:
all_pol_addresses = set(pd.concat([csv2['address']]))

# main_not_in_csv = pd.DataFrame(
#     [address for address in main_addresses if address not in all_csv_addresses],
#     columns=['address']
# )

# arb_not_in_csv = pd.DataFrame(
#     [address for address in arb_addresses if address not in all_csv_addresses],
#     columns=['address']
# )

pol_not_in_csv = pd.DataFrame(
    [address for address in pol_addresses if address not in all_pol_addresses],
    columns=['address']
)

print(len(main_not_in_csv), len(main_addresses))
print(len(arb_not_in_csv), len(arb_addresses))
print(len(pol_not_in_csv), len(pol_addresses))

In [ ]:
pol_not_in_csv_slice = pol_not_in_csv.iloc[:10000]

for i, row in pol_not_in_csv_slice.iterrows():
    print(i)
    address = row['address']
    pol_not_in_csv_slice.at[i, 'type'] = is_contract(address)
    time.sleep(1/25)

pol_not_in_csv_slice.to_csv('../data/addresses/checked_pol_addresses.csv', index=False)

In [ ]:
# Extract all distinct addresses from 'from' and 'to' columns in filtered_df
distinct_filtered_addresses = pd.DataFrame(
    pd.concat([filtered_df['from'], filtered_df['to']]).unique(), 
    columns=['address']
)

# Check how many of these addresses are in csv1, csv2, and csv3
in_csv1 = distinct_filtered_addresses['address'].isin(csv1['address']).sum()
in_csv2 = distinct_filtered_addresses['address'].isin(csv2['address']).sum()
in_csv3 = distinct_filtered_addresses['address'].isin(csv3['address']).sum()

# Find addresses that are not in any of the csv files
not_in_any_csv = distinct_filtered_addresses[
    ~distinct_filtered_addresses['address'].isin(pd.concat([csv1['address'], csv2['address'], csv3['address']]))
]

# Print the results
print(f"Addresses in csv1: {in_csv1}")
print(f"Addresses in csv2: {in_csv2}")
print(f"Addresses in csv3: {in_csv3}")
print(f"Addresses not in any csv: {len(not_in_any_csv)}")

In [5]:
csv_pol = pd.read_csv('../data/addresses/checked_pol_addresses.csv')